# 日付ごと・地域ごとの男性の割合をだす

こういう理由でこの調査を行いますよ。

issueのリンク: http://example.com/

### 期間
6/1 ~ 6/3

## 結果
こんな感じになりました。

最終結果はページ上部にも持ってきておくと、githubのリンクから見にきた人が見やすいと思うのでいつも上に持ってきてます。

### 地域ごと、日付ごとの男性率（%）

In [28]:
pd.DataFrame.from_dict(area2percentage_man, orient='index', columns=dates)

,06/01,06/02,06/03
okinawa,51.56,50.12,52.36
tokyo,50.63,50.91,50.56
osaka,50.63,50.98,50.63
hokkaido,50.85,51.31,49.14


## 考察
なんか書いても良いかも

---
markdownなのでハイフン3つで区切れる




# 以下、結果を出すために書いたスクリプト

In [1]:
# import pandas as pd  # あらかじめimportしてる（上にある記述参照）ので不要
csv_file = "./data/example.csv"

In [2]:
df = pd.read_csv(csv_file)

In [17]:
# 11件だけ表示
# このデータを取得する際、日付、area、genderでgroup byしてるので綺麗に並んでる
df[:11]

,date,area,gender,count
0,2018-06-01,okinawa,0,16187
1,2018-06-01,okinawa,1,17230
2,2018-06-01,tokyo,0,26833
3,2018-06-01,tokyo,1,27517
4,2018-06-01,osaka,0,24596
5,2018-06-01,osaka,1,25226
6,2018-06-01,hokkaido,0,19765
7,2018-06-01,hokkaido,1,20449
8,2018-06-02,okinawa,0,17923
9,2018-06-02,okinawa,1,18011


In [6]:
records = df.to_dict("records")  # 各行を、表のheaderをkey、カラムの中身をvalueとするdictに変換

In [8]:
records[:3]

[{'area': 'okinawa', 'count': 16187, 'date': '2018-06-01', 'gender': 0},
 {'area': 'okinawa', 'count': 17230, 'date': '2018-06-01', 'gender': 1},
 {'area': 'tokyo', 'count': 26833, 'date': '2018-06-01', 'gender': 0}]

#### 最終的には、行が地域、列が日付となるような表にしたい。

from_dictを使えば良さそう。
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.from_dict.html

```
>>> data = {'row_1': [3, 2, 1, 0], 'row_2': ['a', 'b', 'c', 'd']}
>>> pd.DataFrame.from_dict(data, orient='index',
...                        columns=['A', 'B', 'C', 'D'])
       A  B  C  D
row_1  3  2  1  0
row_2  a  b  c  d
```

今回は、ABCDの位置に日付リスト、row_1row_2の位置にareaがくれば良い。日付リストと、keyをarea、valueを日付順の男性率リストとした辞書が作れれば目的の表が作れます。

以下2つの変数を作る

```
dates: ["6/1", "6/2", "6/3"]
area2percentage_man: {"tokyo": [6/1の男性率, 6/2の男性率, 6/3の男性率],...}
```

In [12]:
man_records = records[1::2]
woman_records = records[::2]

In [13]:
man_records[:3]  # 男性のrecordだけになってる（gender: 1 が男性）

[{'area': 'okinawa', 'count': 17230, 'date': '2018-06-01', 'gender': 1},
 {'area': 'tokyo', 'count': 27517, 'date': '2018-06-01', 'gender': 1},
 {'area': 'osaka', 'count': 25226, 'date': '2018-06-01', 'gender': 1}]

In [14]:
len(man_records) == len(woman_records)  # listの長さ同じじゃないとおかしい

True

In [19]:
from collections import defaultdict

In [24]:
dates = []
area2percentage_man = defaultdict(list)  # 新たなkeyができた時にvalueはデフォルトでlistとしてくれる

for (man_record, woman_record) in zip(man_records, woman_records):
    area = man_record['area']
    date = man_record['date']
    shortened_date = "/".join(date.split("-")[1:]) # 2018-06-01 は長いので 06/01 にする
    
    
    # 日付リスト作っておく
    if shortened_date not in dates:
        dates.append(shortened_date)
        
    # 男性率
    # defaultdictのおかげでいきなりappendできる
    total = man_record['count'] + woman_record['count']
    rate = 1.0 * man_record['count'] / total  # 0.5156058293682856 のような形
    percentage = round(rate * 100, 2)  # %表示 （小数第二位まで）
    area2percentage_man[area].append(percentage)

In [25]:
dates

['06/01', '06/02', '06/03']

In [26]:
area2percentage_man

defaultdict(list,
            {'hokkaido': [50.85, 51.31, 49.14],
             'okinawa': [51.56, 50.12, 52.36],
             'osaka': [50.63, 50.98, 50.63],
             'tokyo': [50.63, 50.91, 50.56]})

### 地域ごと、日付ごとの男性率（%）

In [27]:
pd.DataFrame.from_dict(area2percentage_man, orient='index', columns=dates)

,06/01,06/02,06/03
okinawa,51.56,50.12,52.36
tokyo,50.63,50.91,50.56
osaka,50.63,50.98,50.63
hokkaido,50.85,51.31,49.14
